In [1]:
!nvidia-smi

Thu Nov  3 10:51:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -qq tensorflow==2.9.0
!pip install -qq keras==2.9.0

In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
import re

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import itertools

import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC  
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import regularizers

from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow

p_stemmer = PorterStemmer() # initialize PorterStemmer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
URL_BASE = 'https://raw.githubusercontent.com/xiao-yangg/CE4045-NLP/main/'
filenames = ['apple_sentiment_online_filtered.csv', 'combined_dataset.csv', 'test_dataset.csv', 'training_dataset.csv', 'twitter_tweets_filtered.csv', 'us_airline_filtered.csv', 'original label.csv']

In [5]:
tsds = pd.read_csv(URL_BASE + 'test_dataset.csv')
trds = pd.read_csv(URL_BASE + 'training_dataset.csv')

In [6]:
tsds.rename(columns={'final_label': 'sentiment'}, inplace=True)
trds.rename(columns={'final_label': 'sentiment'}, inplace=True)

tsds.drop_duplicates(['tweet'], inplace=True)
trds.drop_duplicates(['tweet'], inplace=True)

In [7]:
# Tokenization
def tokenize(text):
    return word_tokenize(text)

# Stemming
def stemming(token_list):
    stemmed_list = []
    for word in token_list:
        stemmed_list.append(p_stemmer.stem(word))
    return stemmed_list

# Lemmatization
def lemmatization(stemmed_list):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = []
    for word in stemmed_list:
        lemma_list.append(wordnet_lemmatizer.lemmatize(word))
    return lemma_list

# Removal of Stopwords
def remove_stopword(text):
    nltk_tokenList = tokenize(text) # tokenize

    filtered_sentence = [] # sentence without stopwords
    nltk_stop_words = set(stopwords.words("english"))
    for w in nltk_tokenList:  
        if w not in nltk_stop_words:  
            filtered_sentence.append(w)
    
    return ' '.join(filtered_sentence)

# Combination of stemming and lemmatization
def nltk_process(text):
    nltk_tokenList = tokenize(text) # tokenize
    nltk_stemmedList = stemming(nltk_tokenList) # stemmed
    nltk_lemmaList = lemmatization(nltk_stemmedList) # lemma
    return ' '.join(nltk_lemmaList)

In [8]:
def normalize_df(df):    
    df['tweet'] = df['tweet'].apply(remove_stopword) # obtain tweet without stopwords
    df['tweet'] = df['tweet'].apply(nltk_process) # tweet with no stopwords go through stemming and lemma

    # New dataframe of text normalized tweets
    columns_titles = ['tweet','sentiment']
    return df.reindex(columns=columns_titles)

In [9]:
tsds = normalize_df(tsds)
trds = normalize_df(trds)

In [10]:
all_train_tweets = trds['tweet'].copy()
all_train_tweets = all_train_tweets.append(tsds['tweet'])

In [11]:
all_train_tweets.shape

(16504,)

In [12]:
max_words = 8000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(all_train_tweets.values)

In [13]:
def build_datasets(df):
  df_0 = df[df['sentiment'] == 'NEGATIVE']
  df_1 = df[df['sentiment'] == 'NEUTRAL']
  df_2 = df[df['sentiment'] == 'POSITIVE']

  df_0.iloc[:, 1] = 1
  df_1.iloc[:, 1] = 0
  df_2.iloc[:, 1] = 1
  df_SUB = shuffle(pd.concat([df_0, df_1, df_2]).copy())

  df_0.iloc[:, 1] = 0
  df_2.iloc[:, 1] = 1
  df_POL = shuffle(pd.concat([df_0, df_1, df_2]).copy())
  return df_SUB, df_POL, df_1

In [14]:
tsds_SUB, tsds_POL, tsds_NEU = build_datasets(tsds)
trds_SUB, trds_POL, trds_NEU = build_datasets(trds)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
sequences = tokenizer.texts_to_sequences(trds['tweet'].values)
X2 = pad_sequences(sequences, maxlen=max_len)
Y2 = pd.get_dummies(trds['sentiment']).values

sequences = tokenizer.texts_to_sequences(tsds['tweet'].values)
X = pad_sequences(sequences, maxlen=max_len)
Y = pd.get_dummies(tsds['sentiment']).values

X5, X6, Y5, Y6 = train_test_split(X, Y, test_size=0.25)

In [16]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20)) # The embedding layer
model1.add(layers.LSTM(15, dropout=0.1)) # Our LSTM layer
model1.add(layers.Dense(3, activation='softmax'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping1 = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, restore_best_weights=True)

In [17]:
%%time
history = model1.fit(X2, Y2, epochs=100, validation_data=[X6, Y6], callbacks=[earlystopping1])

Epoch 1/100
366/366 [==============================] - 12s 14ms/step - loss: 0.8132 - accuracy: 0.6401 - val_loss: 1.1832 - val_accuracy: 0.5100
Epoch 2/100
366/366 [==============================] - 4s 12ms/step - loss: 0.4725 - accuracy: 0.8309 - val_loss: 1.0838 - val_accuracy: 0.5708
Epoch 3/100
366/366 [==============================] - 5s 14ms/step - loss: 0.3235 - accuracy: 0.8894 - val_loss: 1.0516 - val_accuracy: 0.6692
Epoch 4/100
366/366 [==============================] - 5s 12ms/step - loss: 0.2271 - accuracy: 0.9293 - val_loss: 1.1700 - val_accuracy: 0.6892
Epoch 5/100
366/366 [==============================] - 4s 12ms/step - loss: 0.1723 - accuracy: 0.9475 - val_loss: 1.2564 - val_accuracy: 0.6933
Epoch 6/100
366/366 [==============================] - 5s 12ms/step - loss: 0.1376 - accuracy: 0.9609 - val_loss: 1.4175 - val_accuracy: 0.6992
Epoch 7/100
366/366 [==============================] - 5s 13ms/step - loss: 0.1184 - accuracy: 0.9668 - val_loss: 1.5236 - val_accuracy

In [18]:
%%time
history = model1.fit(X5, Y5, epochs=100, validation_data=[X6, Y6], callbacks=[earlystopping1])

Epoch 1/100
113/113 [==============================] - 2s 20ms/step - loss: 0.9114 - accuracy: 0.6656 - val_loss: 0.7304 - val_accuracy: 0.7025
Epoch 2/100
113/113 [==============================] - 3s 24ms/step - loss: 0.6284 - accuracy: 0.7500 - val_loss: 0.6910 - val_accuracy: 0.7208
Epoch 3/100
113/113 [==============================] - 2s 16ms/step - loss: 0.5120 - accuracy: 0.8042 - val_loss: 0.6850 - val_accuracy: 0.7325
Epoch 4/100
113/113 [==============================] - 2s 15ms/step - loss: 0.4152 - accuracy: 0.8514 - val_loss: 0.6994 - val_accuracy: 0.7300
Epoch 5/100
113/113 [==============================] - 2s 14ms/step - loss: 0.3427 - accuracy: 0.8800 - val_loss: 0.7644 - val_accuracy: 0.7408
Epoch 6/100
113/113 [==============================] - 2s 15ms/step - loss: 0.2852 - accuracy: 0.9006 - val_loss: 0.7816 - val_accuracy: 0.7350
Epoch 7/100
113/113 [==============================] - 2s 16ms/step - loss: 0.2433 - accuracy: 0.9128 - val_loss: 0.8291 - val_accuracy:

In [19]:
model1.evaluate(X6, Y6)

38/38 [==============================] - 1s 14ms/step - loss: 0.7644 - accuracy: 0.7408


[0.7644030451774597, 0.7408333420753479]

In [20]:
# text = 'applewatch hello!'.lower()
# text = remove_stopword(text)
# text = nltk_process(text)
# temp_sequences = tokenizer.texts_to_sequences([text])
# temp_X = pad_sequences(temp_sequences, maxlen=max_len)
# model1.predict(temp_X)

In [21]:
sequences = tokenizer.texts_to_sequences(trds_SUB['tweet'].values)
X2_SUB = pad_sequences(sequences, maxlen=max_len)
Y2_SUB = pd.get_dummies(trds_SUB['sentiment']).values

sequences = tokenizer.texts_to_sequences(tsds_SUB['tweet'].values)
X_SUB = pad_sequences(sequences, maxlen=max_len)
Y_SUB = pd.get_dummies(tsds_SUB['sentiment']).values

X5_SUB, X6_SUB, Y5_SUB, Y6_SUB = train_test_split(X_SUB, Y_SUB, test_size=0.25)

In [22]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 20)) # The embedding layer
model2.add(layers.Bidirectional(layers.LSTM(15, dropout=0.1))) # Our LSTM layer
model2.add(layers.Dense(2, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping2 = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

In [23]:
%%time
history = model2.fit(X2_SUB, Y2_SUB, epochs=100, validation_data=[X6_SUB, Y6_SUB], callbacks=[earlystopping1])

Epoch 1/100
366/366 [==============================] - 14s 24ms/step - loss: 0.5112 - accuracy: 0.7427 - val_loss: 0.5929 - val_accuracy: 0.7250
Epoch 2/100
366/366 [==============================] - 8s 21ms/step - loss: 0.2371 - accuracy: 0.9159 - val_loss: 0.6422 - val_accuracy: 0.7625
Epoch 3/100
366/366 [==============================] - 9s 25ms/step - loss: 0.1528 - accuracy: 0.9525 - val_loss: 0.7551 - val_accuracy: 0.7758
Epoch 4/100
366/366 [==============================] - 8s 21ms/step - loss: 0.1182 - accuracy: 0.9643 - val_loss: 0.9145 - val_accuracy: 0.7817
Epoch 5/100
366/366 [==============================] - 8s 21ms/step - loss: 0.1008 - accuracy: 0.9706 - val_loss: 0.8678 - val_accuracy: 0.7908
Epoch 6/100
366/366 [==============================] - 7s 20ms/step - loss: 0.0881 - accuracy: 0.9751 - val_loss: 1.0741 - val_accuracy: 0.7925
Epoch 7/100
366/366 [==============================] - 8s 21ms/step - loss: 0.0821 - accuracy: 0.9760 - val_loss: 1.0518 - val_accuracy

In [24]:
%%time
history = model2.fit(X5_SUB, Y5_SUB, epochs=100, validation_data=[X6_SUB, Y6_SUB], callbacks=[earlystopping1])

Epoch 1/100
113/113 [==============================] - 3s 24ms/step - loss: 0.5092 - accuracy: 0.7964 - val_loss: 0.4022 - val_accuracy: 0.8258
Epoch 2/100
113/113 [==============================] - 3s 23ms/step - loss: 0.3242 - accuracy: 0.8653 - val_loss: 0.4079 - val_accuracy: 0.8283
Epoch 3/100
113/113 [==============================] - 3s 23ms/step - loss: 0.2684 - accuracy: 0.8983 - val_loss: 0.4223 - val_accuracy: 0.8275
Epoch 4/100
113/113 [==============================] - 3s 23ms/step - loss: 0.2195 - accuracy: 0.9192 - val_loss: 0.4645 - val_accuracy: 0.8225
Epoch 4: early stopping
CPU times: user 11.6 s, sys: 920 ms, total: 12.6 s
Wall time: 11.5 s


In [25]:
model2.evaluate(X6_SUB, Y6_SUB)

38/38 [==============================] - 0s 12ms/step - loss: 0.4079 - accuracy: 0.8283


[0.4078553020954132, 0.82833331823349]

In [26]:
sequences = tokenizer.texts_to_sequences(trds_POL['tweet'].values)
X2_POL = pad_sequences(sequences, maxlen=max_len)
Y2_POL = pd.get_dummies(trds_POL['sentiment']).values

sequences = tokenizer.texts_to_sequences(tsds_POL['tweet'].values)
X_POL = pad_sequences(sequences, maxlen=max_len)
Y_POL = pd.get_dummies(tsds_POL['sentiment']).values

X5_POL, X6_POL, Y5_POL, Y6_POL = train_test_split(X_POL, Y_POL, test_size=0.25)

In [27]:
model3 = Sequential()
model3.add(layers.Embedding(max_words, 20)) # The embedding layer
model3.add(layers.Bidirectional(layers.LSTM(15, dropout=0.1))) # Our LSTM layer
model3.add(layers.Dense(2, activation='softmax'))
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping3 = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

In [28]:
%%time
history = model3.fit(X2_POL, Y2_POL, epochs=100, validation_data=[X6_POL, Y6_POL], callbacks=[earlystopping1])

Epoch 1/100
366/366 [==============================] - 12s 23ms/step - loss: 0.5213 - accuracy: 0.7245 - val_loss: 0.6819 - val_accuracy: 0.7250
Epoch 2/100
366/366 [==============================] - 7s 20ms/step - loss: 0.2531 - accuracy: 0.9080 - val_loss: 0.7612 - val_accuracy: 0.7317
Epoch 3/100
366/366 [==============================] - 7s 20ms/step - loss: 0.1791 - accuracy: 0.9400 - val_loss: 0.7361 - val_accuracy: 0.7433
Epoch 4/100
366/366 [==============================] - 9s 24ms/step - loss: 0.1471 - accuracy: 0.9526 - val_loss: 0.9712 - val_accuracy: 0.7575
Epoch 5/100
366/366 [==============================] - 7s 20ms/step - loss: 0.1232 - accuracy: 0.9613 - val_loss: 0.9519 - val_accuracy: 0.7650
Epoch 6/100
366/366 [==============================] - 8s 21ms/step - loss: 0.1046 - accuracy: 0.9678 - val_loss: 1.0607 - val_accuracy: 0.7608
Epoch 7/100
366/366 [==============================] - 7s 20ms/step - loss: 0.0908 - accuracy: 0.9721 - val_loss: 1.1236 - val_accuracy

In [29]:
%%time
history = model3.fit(X5_POL, Y5_POL, epochs=100, validation_data=[X6_POL, Y6_POL], callbacks=[earlystopping1])

Epoch 1/100
113/113 [==============================] - 3s 23ms/step - loss: 0.5921 - accuracy: 0.7628 - val_loss: 0.4991 - val_accuracy: 0.7717
Epoch 2/100
113/113 [==============================] - 3s 23ms/step - loss: 0.3886 - accuracy: 0.8267 - val_loss: 0.4975 - val_accuracy: 0.7842
Epoch 3/100
113/113 [==============================] - 3s 23ms/step - loss: 0.3034 - accuracy: 0.8711 - val_loss: 0.5183 - val_accuracy: 0.7825
Epoch 4/100
113/113 [==============================] - 3s 23ms/step - loss: 0.2396 - accuracy: 0.9025 - val_loss: 0.5369 - val_accuracy: 0.7917
Epoch 5/100
113/113 [==============================] - 3s 23ms/step - loss: 0.1890 - accuracy: 0.9264 - val_loss: 0.5838 - val_accuracy: 0.7900
Epoch 6/100
113/113 [==============================] - 3s 25ms/step - loss: 0.1508 - accuracy: 0.9447 - val_loss: 0.6138 - val_accuracy: 0.7850
Epoch 6: early stopping
CPU times: user 17.2 s, sys: 1.31 s, total: 18.5 s
Wall time: 15.8 s


In [30]:
model3.evaluate(X6_POL, Y6_POL)

38/38 [==============================] - 1s 15ms/step - loss: 0.5369 - accuracy: 0.7917


[0.5368744134902954, 0.7916666865348816]